In [1]:
import numpy as np
import pandas as pd

import torch

In [2]:
from pts.dataset import to_pandas, MultivariateGrouper, TrainDatasets
from pts.dataset.repository import get_dataset, dataset_recipes
from pts.model.tempflow import TempFlowEstimator
from pts.model.transformer_tempflow import TransformerTempFlowEstimator
from pts import Trainer
from pts.evaluation import make_evaluation_predictions
from pts.evaluation import MultivariateEvaluator

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Prepeare data set

In [4]:
dataset = get_dataset("traffic_nips", regenerate=False)

In [5]:
dataset.metadata

MetaData(freq='H', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='963')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=24)

In [6]:
train_grouper = MultivariateGrouper(max_target_dim=int(dataset.metadata.feat_static_cat[0].cardinality))

test_grouper = MultivariateGrouper(num_test_dates=int(len(dataset.test)/len(dataset.train)), 
                                   max_target_dim=int(dataset.metadata.feat_static_cat[0].cardinality))

In [7]:
dataset_train = train_grouper(dataset.train)
dataset_test = test_grouper(dataset.test)

## Evaluator

In [8]:
evaluator = MultivariateEvaluator(quantiles=(np.arange(20)/20.0)[1:],
                                  target_agg_funcs={'sum': np.sum})

## `GRU-Real-NVP`

In [9]:
estimator = TempFlowEstimator(
    target_dim=int(dataset.metadata.feat_static_cat[0].cardinality),
    prediction_length=dataset.metadata.prediction_length,
    cell_type='GRU',
    input_size=3856,
    num_cells=128,
    freq=dataset.metadata.freq,
    scaling=True,
    n_blocks=4,
    dropout_rate=0.3,
    trainer=Trainer(device=device,
                    epochs=21,
                    learning_rate=1e-3,
                    num_batches_per_epoch=100,
                    batch_size=32,
                    num_workers=8)
)

In [10]:
predictor = estimator.train(dataset_train)
forecast_it, ts_it = make_evaluation_predictions(dataset=dataset_test,
                                             predictor=predictor,
                                             num_samples=100)
forecasts = list(forecast_it)
targets = list(ts_it)

agg_metric, _ = evaluator(targets, forecasts, num_series=len(dataset_test))

99it [00:25,  3.81it/s, avg_epoch_loss=65.3, epoch=0]  
99it [00:25,  3.81it/s, avg_epoch_loss=-358, epoch=1]
99it [00:24,  3.97it/s, avg_epoch_loss=-504, epoch=2]
99it [00:25,  3.94it/s, avg_epoch_loss=-598, epoch=3]
99it [00:25,  3.94it/s, avg_epoch_loss=-656, epoch=4]
99it [00:25,  3.90it/s, avg_epoch_loss=-672, epoch=5]
99it [00:25,  3.91it/s, avg_epoch_loss=-727, epoch=6]
99it [00:24,  3.97it/s, avg_epoch_loss=-763, epoch=7]
99it [00:25,  3.94it/s, avg_epoch_loss=-787, epoch=8]
99it [00:25,  3.94it/s, avg_epoch_loss=-808, epoch=9]
99it [00:24,  3.97it/s, avg_epoch_loss=-815, epoch=10]
99it [00:25,  3.94it/s, avg_epoch_loss=-847, epoch=11]
99it [00:25,  3.92it/s, avg_epoch_loss=-861, epoch=12]
99it [00:25,  3.92it/s, avg_epoch_loss=-879, epoch=13]
99it [00:24,  3.97it/s, avg_epoch_loss=-889, epoch=14]
99it [00:24,  3.97it/s, avg_epoch_loss=-892, epoch=15]
99it [00:24,  3.99it/s, avg_epoch_loss=-916, epoch=16]
99it [00:24,  3.97it/s, avg_epoch_loss=-925, epoch=17]
99it [00:24,  3.98

### Metrics

In [11]:
print("CRPS: {}".format(agg_metric['mean_wQuantileLoss']))
print("ND: {}".format(agg_metric['ND']))
print("NRMSE: {}".format(agg_metric['NRMSE']))
print("MSE: {}".format(agg_metric['MSE']))

CRPS: 0.24719307509342245
ND: 0.3014458288816536
NRMSE: 0.5914382112230867
MSE: 0.0009593914131918332


In [12]:
print("CRPS-Sum: {}".format(agg_metric['m_sum_mean_wQuantileLoss']))
print("ND-Sum: {}".format(agg_metric['m_sum_ND']))
print("NRMSE-Sum: {}".format(agg_metric['m_sum_NRMSE']))
print("MSE-Sum: {}".format(agg_metric['m_sum_MSE']))
print("ES: {}".format(agg_metric['m_ES']))

CRPS-Sum: 0.15058280199691917
ND-Sum: 0.18837051658426468
NRMSE-Sum: 0.2667453643788735
MSE-Sum: 180.9769726707822
ES: 0.6409833473818642


## `GRU-MAF`

In [13]:
estimator = TempFlowEstimator(
    target_dim=int(dataset.metadata.feat_static_cat[0].cardinality),
    prediction_length=dataset.metadata.prediction_length,
    cell_type='GRU',
    num_cells=128,
    input_size=3856,
    freq=dataset.metadata.freq,
    scaling=True,
    flow_type='MAF',
    dropout_rate=0.3,
    trainer=Trainer(device=device,
                    epochs=60,
                    learning_rate=1e-3,
                    num_batches_per_epoch=100,
                    batch_size=32,
                    num_workers=6)
)

In [ ]:
predictor = estimator.train(dataset_train)
forecast_it, ts_it = make_evaluation_predictions(dataset=dataset_test,
                                             predictor=predictor,
                                             num_samples=100)
forecasts = list(forecast_it)
targets = list(ts_it)

agg_metric, _ = evaluator(targets, forecasts, num_series=len(dataset_test))

99it [00:24,  4.07it/s, avg_epoch_loss=245, epoch=0]    
99it [00:25,  3.95it/s, avg_epoch_loss=-146, epoch=1] 
99it [00:23,  4.13it/s, avg_epoch_loss=-331, epoch=2]
99it [00:24,  4.10it/s, avg_epoch_loss=-450, epoch=3]
99it [00:23,  4.15it/s, avg_epoch_loss=-527, epoch=4]
99it [00:23,  4.18it/s, avg_epoch_loss=-582, epoch=5]
99it [00:24,  4.05it/s, avg_epoch_loss=-627, epoch=6]
99it [00:24,  4.10it/s, avg_epoch_loss=-663, epoch=7]
99it [00:25,  3.93it/s, avg_epoch_loss=-695, epoch=8]
99it [00:23,  4.18it/s, avg_epoch_loss=-723, epoch=9]
99it [00:24,  4.10it/s, avg_epoch_loss=-747, epoch=10]
99it [00:24,  4.12it/s, avg_epoch_loss=-770, epoch=11]
99it [00:24,  4.07it/s, avg_epoch_loss=-788, epoch=12]
99it [00:25,  3.96it/s, avg_epoch_loss=-806, epoch=13]
99it [00:24,  4.03it/s, avg_epoch_loss=-822, epoch=14]
99it [00:24,  3.97it/s, avg_epoch_loss=-836, epoch=15]
99it [00:24,  4.06it/s, avg_epoch_loss=-850, epoch=16]
99it [00:24,  3.99it/s, avg_epoch_loss=-861, epoch=17]
99it [00:24,  4.

### Metrics

In [13]:
print("CRPS: {}".format(agg_metric['mean_wQuantileLoss']))
print("ND: {}".format(agg_metric['ND']))
print("NRMSE: {}".format(agg_metric['NRMSE']))
print("MSE: {}".format(agg_metric['MSE']))

CRPS: 0.14656428154884796
ND: 0.18578166848226196
NRMSE: 0.458467186514283
MSE: 0.0005764923958606087


In [14]:
print("CRPS-Sum: {}".format(agg_metric['m_sum_mean_wQuantileLoss']))
print("ND-Sum: {}".format(agg_metric['m_sum_ND']))
print("NRMSE-Sum: {}".format(agg_metric['m_sum_NRMSE']))
print("MSE-Sum: {}".format(agg_metric['m_sum_MSE']))
print("ES: {}".format(agg_metric['m_ES']))

CRPS-Sum: 0.08093098952066129
ND-Sum: 0.10452289564790243
NRMSE-Sum: 0.15947451943817495
MSE-Sum: 64.686277843657


## `Transformer-MAF`

In [17]:
estimator = TransformerTempFlowEstimator(
    d_model=64,
    input_size=3856,
    target_dim=int(dataset.metadata.feat_static_cat[0].cardinality),
    prediction_length=dataset.metadata.prediction_length,
    flow_type='MAF',
    freq=dataset.metadata.freq,
    trainer=Trainer(
        device=device,
        epochs=35,
        learning_rate=1e-3,
        num_batches_per_epoch=100,
        batch_size=64,
    )
)

In [18]:
predictor = estimator.train(dataset_train)
forecast_it, ts_it = make_evaluation_predictions(dataset=dataset_test,
                                             predictor=predictor,
                                             num_samples=100)
forecasts = list(forecast_it)
targets = list(ts_it)

agg_metric, _ = evaluator(targets, forecasts, num_series=len(dataset_test))

99it [00:46,  2.13it/s, avg_epoch_loss=345, epoch=0]    
99it [00:45,  2.17it/s, avg_epoch_loss=-71.1, epoch=1]
99it [00:45,  2.18it/s, avg_epoch_loss=-280, epoch=2]
99it [00:44,  2.20it/s, avg_epoch_loss=-413, epoch=3]
99it [00:45,  2.16it/s, avg_epoch_loss=-503, epoch=4]
99it [00:45,  2.18it/s, avg_epoch_loss=-569, epoch=5]
99it [00:46,  2.12it/s, avg_epoch_loss=-620, epoch=6]
99it [00:46,  2.11it/s, avg_epoch_loss=-662, epoch=7]
99it [00:45,  2.17it/s, avg_epoch_loss=-698, epoch=8]
99it [00:45,  2.18it/s, avg_epoch_loss=-729, epoch=9]
99it [00:45,  2.16it/s, avg_epoch_loss=-755, epoch=10]
99it [00:45,  2.19it/s, avg_epoch_loss=-778, epoch=11]
99it [00:46,  2.12it/s, avg_epoch_loss=-799, epoch=12]
99it [00:45,  2.15it/s, avg_epoch_loss=-819, epoch=13]
99it [00:45,  2.16it/s, avg_epoch_loss=-836, epoch=14]
99it [00:45,  2.18it/s, avg_epoch_loss=-851, epoch=15]
99it [00:45,  2.17it/s, avg_epoch_loss=-866, epoch=16]
99it [00:45,  2.16it/s, avg_epoch_loss=-881, epoch=17]
99it [00:45,  2.

### Metrics

In [19]:
print("CRPS: {}".format(agg_metric['mean_wQuantileLoss']))
print("ND: {}".format(agg_metric['ND']))
print("NRMSE: {}".format(agg_metric['NRMSE']))
print("MSE: {}".format(agg_metric['MSE']))

CRPS: 0.13711744408321852
ND: 0.17257855543080755
NRMSE: 0.44924711940316436
MSE: 0.0005535382926495773


In [20]:
print("CRPS-Sum: {}".format(agg_metric['m_sum_mean_wQuantileLoss']))
print("ND-Sum: {}".format(agg_metric['m_sum_ND']))
print("NRMSE-Sum: {}".format(agg_metric['m_sum_NRMSE']))
print("MSE-Sum: {}".format(agg_metric['m_sum_MSE']))
print("ES: {}".format(agg_metric['m_ES']))

CRPS-Sum: 0.07060320911475239
ND-Sum: 0.08772725442330981
NRMSE-Sum: 0.12958588059679835
MSE-Sum: 42.711510067894345
